In [1]:
from datetime import datetime, timedelta

In [2]:
d1 = datetime.now()

In [3]:
d1

datetime.datetime(2021, 12, 25, 5, 7, 32, 243988)

In [4]:
d2 = datetime.now()

In [5]:
dt = d2-d1
print(int(dt.total_seconds()))

2


In [6]:
def field(s,start,size): return s[start:start+size]
def Int(s):
    return sum(c << (i * 8) for i, c in enumerate(s))

In [7]:
f=open('\\\\.\\PHYSICALDRIVE0','rb')

OSError: [Errno 5] Input/output error: '\\\\.\\PHYSICALDRIVE0'

In [3]:
mbr =   f.read(512)

In [4]:
part0 = field(mbr,0x1be,16)

In [5]:
Int(field(part0,8,4))

63

In [6]:
Int(field(part0,12,4))*512/1024/1024/1024

195.30924654006958

In [7]:
Int(field(field(mbr,0x1be+16,16),8,4))*512/1024/1024/1024

195.3093056678772

In [8]:
Int(field(field(mbr,0x1be+16,16),12,4))*512/1024/1024/1024

736.2033896446228

In [9]:
def mainPartitions(mbr):
    assert field(mbr,0x01FE,2)==b'\x55\xaa', 'mbr signature absent'
    def p(pe):
        status = field(pe,0,1)
        type = field(pe,4,1)
        return (Int(field(pe,8,4)),Int(field(pe,12,4)),
                True if status==b'\x80' else 
                False if status==b'\x00' else status,
                'EBR' if type==b'\x0f' else
                'EBR' if type==b'\x05' else
                'NTFS/exFAT' if type==b'\x07' else
                'Linux file system' if type==b'\x83' else
                'Linux swap space' if type==b'\x82' else
                type
               )
    return (
        p(field(mbr,0x1be+16*0,16)),
        p(field(mbr,0x1be+16*1,16)),
        p(field(mbr,0x1be+16*2,16)),
        p(field(mbr,0x1be+16*3,16)),
    )

In [10]:
mainPartitions(mbr)

((63, 409593177, True, 'NTFS/exFAT'),
 (409593301, 1543930411, False, 'EBR'),
 (0, 0, False, b'\x00'),
 (0, 0, False, b'\x00'))

In [11]:
def getSector(f,i):
    f.seek(i*512);
    return f.read(512)
def logicalPartitions(f,start):
    partitions = []
    next = start
    while next!=-1:
        #print(next)
        ebr = getSector(f,next)
        (part,nn,x,y) = mainPartitions(ebr)
        part =  (part[0]+next,*part[1:],part[0])
        partitions.append(part)
        if nn[1]==0: next = -1
        else: next = nn[0]+start
    return partitions       


In [12]:
logicalPartitions(f,409593301)

[(409593856, 835704832, False, 'NTFS/exFAT', 555),
 (1245300736, 424314880, False, 'NTFS/exFAT', 2048),
 (1669632000, 83050496, False, 'Linux file system', 16384),
 (1752684544, 85016576, False, 'Linux file system', 2048),
 (1837705216, 111693824, False, 'Linux file system', 4096),
 (1949403136, 4120576, False, 'Linux swap space', 4096)]

In [13]:
vbr = getSector(f,409593856)

In [14]:
field(vbr,3,8)

b'NTFS    '

In [15]:
#Bytes per sector
Int(field(vbr,0xb,2))

512

In [16]:
#Sectors Per Cluster
Int(field(vbr,0xd,1))

8

In [17]:
#Media Descriptor
field(vbr,0x15,1)

b'\xf8'

In [18]:
#Sectors Per Track
Int(field(vbr,0x18,2))

63

In [19]:
#Number Of Heads
Int(field(vbr,0x1a,2))

255

In [20]:
#Hidden Sectors 	The number of sectors preceding the partition
Int(field(vbr,0x1c,4))

555

In [21]:
#Total sectors
Int(field(vbr,0x28,8))

835704831

In [22]:
#$MFT cluster number
Int(field(vbr,0x30,8))

786432

In [23]:
#$MFTMirr cluster number
Int(field(vbr,0x38,8))

2

In [24]:
#Bytes or Clusters Per File Record Segment
2**(256-Int(field(vbr,0x40,1)))

1024

In [25]:
#Bytes or Clusters Per Index Buffer
Int(field(vbr,0x44,1))

1

In [26]:
#Volume Serial Number
field(vbr,0x48,8)

b'\xe8MD\x82mD\x82\xdc'

In [27]:
field(vbr,0x1fe,2)

b'U\xaa'